In [1]:
#Setting up selenium webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd

import os
import glob
from time import sleep
import argparse
import math
import os

In [6]:
parser = argparse.ArgumentParser()

parser.add_argument(
    '-ch',
    "--chromedriver_path",
    required=True,
    type=str,
    default='./Data/chrome_driver/chromedriver',
    help="Local path where you put the list of streets",
)

parser.add_argument(
    '-f',
    "--file_path",
    required=True,
    type=str,
    default='./Data/London_streets.csv',
    help="Local path where you put the list of streets",
)

parser.add_argument(
    '-c',
    "--city",
    required=True,
    type=str,
    default='London',
    help="The city where the streets belong to",
)

parser.add_argument(
    '-s',
    "--save_path",
    required=True,
    type=str,
    default='./Data/Reviews/TripAdvisor',
    help="local path where you want to save your scraped data",
)
parser.add_argument(
    '-u',
    "--unfound_path",
    required=True,
    type=str,
    default='./Data/Reviews/TripAdvisor/unfound_streets_reviews_TripAdvisor.txt',
    help="Record unfound streets or streets without reviews",
)
    # args = parser.parse_args(args=[])
cmd_args = ['-ch', './Data/chrome_driver/chromedriver', '-f', 'Data/nyc_streets.csv',
                '-c', 'Singapore', '-s', './Data/Reviews/Trip', '-u', './Data/Reviews/TripAdvisor/unfound_streets_reviews_Trip.txt']
args = parser.parse_args(cmd_args)

In [20]:
# set constant
MAX_WAIT = 20
# initialize webdriver
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('--incognito')  # 隐身模式（无痕模式）
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
# driver = webdriver.Chrome(executable_path=, options=options)
ser = Service(f"{args.chromedriver_path}")
driver = webdriver.Chrome(service=ser, options=options)
# load street names
streets_df = pd.read_csv(args.file_path)
streets = list(streets_df.Street.unique()) 
# read and concatenate csv files
csv_files = glob.glob(args.save_path+'/*.csv')
data_frames = []
street = "Orchard Road"

driver.execute_script("window.open('');")
driver.switch_to.window(driver.window_handles[1])
driver.get('https://sg.trip.com/?locale=en-sg')

wait = WebDriverWait(driver, MAX_WAIT)
sleep(5)
try:
    # search = wait.until(EC.element_to_be_clickable((By.XPATH, './/span[@class="QLiHN o W"]')))
    # search.click()
    box = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ibuHeaderSearch"]/div/div/div[1]/input')))
except TimeoutException:
    print('Cannot locate search box')
sleep(3)
street_name = street +', '+ args.city
print(street_name)
box.send_keys(street_name)
sleep(4)
# search.send_keys(Keys.RETURN)
# sleep(3)
#Clicking to search
driver.find_element(By.XPATH, '//*[@id="ibuHeaderSearch"]/div/div/div[2]/div/div').click()
try:
    # search = wait.until(EC.element_to_be_clickable((By.XPATH, './/span[@class="QLiHN o W"]')))
    # search.click()
    box = wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body")))
    html_content = box.get_attribute('innerHTML')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
except TimeoutException:
    print('FAILLLL')

# Execute JavaScript code to locate the element inside the Shadow DOM
# element_in_shadow_dom = driver.execute_script("return arguments[0].shadowRoot.querySelector('my-element')", web_component)
# driver.find_element(By.XPATH, f'//*[@id="react-tabs-16581"]/div/div[2]//*[contains(translate(text(), "ABCDEFGHIJKLMNOPQRSTUVWXYZ", "abcdefghijklmnopqrstuvwxyz"), "{street_name}")]').click()
driver.close()

Orchard Road, Singapore
